In [2]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.document_loaders import DirectoryLoader

directory = 'C:/Users/farha/Desktop/Farhaan/OwnGPT/files'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)
print(len(documents))

1


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    txt_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = txt_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

8


In [5]:
print(docs[3].page_content)

The mission objectives of Chandrayaan-3 are:

1. To demonstrate Safe and Soft Landing on Lunar Surface 2. To demonstrate Rover roving on the moon and 3. To conduct in-situ scientific experiments.

To achieve the mission objectives, several advanced technologies are present in Lander such as,

1. Altimeters: Laser & RF based Altimeters 2. Velocimeters: Laser Doppler Velocimeter & Lander Horizontal Velocity Camera

3.

Inertial Measurement: Laser Gyro based Inertial referencing and Accelerometer package

4. Propulsion System: 800N Throttleable Liquid Engines, 58N attitude thrusters &

Throttleable Engine Control Electronics

5. Navigation, Guidance & Control (NGC): Powered Descent Trajectory design and

associate software elements

6. Hazard Detection and Avoidance: Lander Hazard Detection & Avoidance Camera

and Processing Algorithm 7. Landing Leg Mechanism.


In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(tiktoken_model_name="ada")
query_result = embeddings.embed_query("Hello World")
print(len(query_result))

1536


In [7]:
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment="asia-southeast1-gcp-free"
)

index_name = "owngpt-index"
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)


c:\Users\farha\anaconda3\envs\testenv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
#index.delete(delete_all=True, namespace='')

In [9]:
def get_similar_docs(query, k=2, score=False):
    if score:
        similar_docs = index.similarity_search_with_score(query, k=k)
    else:
        similar_docs = index.similarity_search(query, k=k)
    return similar_docs

query = "What is the goal of chandrayaan 3?"
similar_docs = get_similar_docs(query)
print(similar_docs)

[Document(page_content='(APXS) and Laser\n\nChandrayaan-3 consists of an indigenous Lander module (LM), Propulsion module (PM) and a Rover with an objective of developing and demonstrating new technologies required for Inter planetary missions. The Lander will have the capability to soft land at a specified lunar site and deploy the Rover which will carry out in-situ chemical analysis of the lunar surface during the course of its mobility. The Lander and the Rover have scientific payloads to carry out experiments on the lunar surface. The main function of PM is to carry the LM from launch vehicle injection till final lunar 100 km circular polar orbit and separate the LM from PM. Apart from this, the Propulsion Module also has one scientific payload as a value addition which will be operated post separation of Lander Module. The launcher identified for Chandrayaan-3 is LVM3 M4 which will place the integrated module in an Elliptic Parking Orbit (EPO) of size ~170 x 36500 km.', metadata={

In [10]:
from langchain.llms import OpenAI
model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name)

c:\Users\farha\anaconda3\envs\testenv\lib\site-packages\langchain\llms\openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\farha\anaconda3\envs\testenv\lib\site-packages\langchain\llms\openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [13]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
    similar_docs = get_similar_docs(query)
    #print(similar_docs)
    answer = chain.run(input_documents=similar_docs, question=query)
    return answer

query = "What is the goal of chandrayaan 3?"
ans = get_answer(query) 
print(ans)

The goal of Chandrayaan-3 is to develop and demonstrate new technologies required for interplanetary missions, including soft landing on a specified lunar site and conducting in-situ chemical analysis of the lunar surface.


In [14]:
query = input("Enter the query(To end: \'x\'): ")
while query != "x":
    print(get_answer(query))
    query = input("Enter the query(To end: \'x\'): ")

The goal of Chandrayaan-3 is to develop and demonstrate new technologies required for interplanetary missions, specifically focusing on soft landing at a specified lunar site, in-situ chemical analysis of the lunar surface, and conducting scientific experiments on the lunar surface.
